In [4]:
import os
import nibabel as nib
import numpy as np
import glob

In [3]:
!pip install nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.0 MB/s eta 0:00:0000:0100:01


In [6]:
os.chdir('/gpfs/space/home/joonas97/data/kits21/kits21/data/')

In [7]:
os.getcwd()

'/gpfs/space/home/joonas97/data/kits21/kits21/data'

In [8]:
def classify_slices(segmentation):
    # give me slice indexes where label 1 (kidney) is present and where the area of kidney is bigger than 1000 pixels
    # currently applying this 1000 pixel filter because im not sure if model can detect very small slices of kidneys
    kidney_slice_indexes = [slice_nr for slice_nr,i in enumerate(segmentation) if 1 in i and np.unique(i, return_counts=True)[1][1] > 1000]
    no_kidney_slice_indexes = [slice_nr for slice_nr,i in enumerate(segmentation) if 1 not in i]
    return kidney_slice_indexes, no_kidney_slice_indexes

In [9]:
segmentations = []
for filename in glob.glob('*/aggregated_MAJ_seg.nii.gz'):
    segmentations.append(filename)

In [10]:
organized_slices = {}
for i, file in enumerate(segmentations[:50]):

    if i % 5 == 0:
        print(i)
    case_id = file.split('/')[0]
    segmentation = nib.load(file).get_fdata()
    kidney_slices, no_kidney_slices =classify_slices(segmentation)
    organized_slices[case_id] = [kidney_slices, no_kidney_slices]

0
5
10
15
20
25
30
35
40
45


In [11]:
len(organized_slices)

50

In [12]:
firstiter = True
for i, (keys, values) in enumerate(organized_slices.items()):
    ct_scan = nib.load(keys+'/imaging.nii.gz').get_fdata()
    if i % 5 == 0:
        print(i)
    if firstiter:
        all_kidney_slices = ct_scan[values[0]]
        all_no_kidney_slices = ct_scan[values[1]]
        firstiter = False
    else:
        new_kidney_slices = ct_scan[values[0]]
        new_no_kidney_slices = ct_scan[values[1]]
        all_kidney_slices = np.concatenate((all_kidney_slices, new_kidney_slices), axis = 0)
        all_no_kidney_slices = np.concatenate((all_no_kidney_slices, new_no_kidney_slices), axis = 0)

0
5


MemoryError: 

In [ ]:
all_kidney_slices.shape

In [10]:
all_no_kidney_slices.shape

(7038, 512, 512)

In [11]:
np.save('kidney_slices',all_kidney_slices)
np.save('no_kidney_slices', all_no_kidney_slices)

In [12]:
test = np.load('kidney_slices.npy')

In [13]:
test.shape

(3463, 512, 512)